# Synthea self-harm classification project: Creating the working datafile

## **See the README file for information on generating the initial datafiles**

The code below is used to merge the initial datafiles into one working datafile (final_data_modified.csv). Specific variables likely to contain information relevant to identifying suicide-related visits were retained in the merged datafile.

In [1]:
import numpy as np
import os
import pandas as pd

# Load the first data file
data_file1 = '/Users/SarahA/synthea/output/csv/encounters.csv'
df1 = pd.read_csv(data_file1, low_memory = False)

# Define the specific variables (columns) to keep
desired_columns = ['Id', 'PATIENT', 'CODE', 'DESCRIPTION', 'REASONCODE', 'REASONDESCRIPTION', 'ENCOUNTERCLASS']

# Filter the dataframe to retain only the desired columns
df1 = df1[desired_columns]

# Load the second data file
data_file2 = '/Users/SarahA/synthea/output/csv/patients.csv'
df2 = pd.read_csv(data_file2, low_memory = False)

# Define the specific variables (columns) to keep
desired_columns = ['Id', 'BIRTHDATE', 'RACE', 'ETHNICITY', 'GENDER', 'INCOME']

# Filter the dataframe to retain only the desired columns
df2 = df2[desired_columns]

# Load the third data file
data_file3 = '/Users/SarahA/synthea/output/csv/claims.csv'
df3 = pd.read_csv(data_file3)

# Define the specific variables (columns) to keep
desired_columns = ['Id', 'PATIENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3', 'DIAGNOSIS4', 'DIAGNOSIS5', 'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8']

# Filter the dataframe to retain only the desired columns
df3 = df3[desired_columns]

In [2]:
print(df1.columns)
print(df2.columns)
print(df3.columns)

Index(['Id', 'PATIENT', 'CODE', 'DESCRIPTION', 'REASONCODE',
       'REASONDESCRIPTION', 'ENCOUNTERCLASS'],
      dtype='object')
Index(['Id', 'BIRTHDATE', 'RACE', 'ETHNICITY', 'GENDER', 'INCOME'], dtype='object')
Index(['Id', 'PATIENTID', 'DIAGNOSIS1', 'DIAGNOSIS2', 'DIAGNOSIS3',
       'DIAGNOSIS4', 'DIAGNOSIS5', 'DIAGNOSIS6', 'DIAGNOSIS7', 'DIAGNOSIS8'],
      dtype='object')


In [3]:
# Merge the first two dataframes
merged_df = df1.merge(df2, left_on='PATIENT', right_on='Id', how='left')

# Rename the 'PATIENTID' column in the third dataframe to match the merged dataframe
df3 = df3.rename(columns={'PATIENTID': 'PATIENT'})

# Merge the third dataframe based on the columns from the first merge
merged_df = merged_df.merge(df3, left_on='PATIENT', right_on='PATIENT', how='left')

merged_df.to_csv('merged_data.csv', index=False)

In [18]:
import pandas as pd

file_path = '/Users/SarahA/synthea/merged_data.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

print(df.head())

/var/folders/dw/9kv692xj65d8csvjp0w67g740000gp/T/ipykernel_51516/1185204573.py:6: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


                                   Id_x                               PATIENT  \
0  5f87dd8b-f3f5-f5ff-b602-629ff78f9dd2  c546d63c-62e1-1322-3628-a9646839e090   
1  5f87dd8b-f3f5-f5ff-b602-629ff78f9dd2  c546d63c-62e1-1322-3628-a9646839e090   
2  5f87dd8b-f3f5-f5ff-b602-629ff78f9dd2  c546d63c-62e1-1322-3628-a9646839e090   
3  5f87dd8b-f3f5-f5ff-b602-629ff78f9dd2  c546d63c-62e1-1322-3628-a9646839e090   
4  5f87dd8b-f3f5-f5ff-b602-629ff78f9dd2  c546d63c-62e1-1322-3628-a9646839e090   

        CODE                                 DESCRIPTION  REASONCODE  \
0  162673000  General examination of patient (procedure)         NaN   
1  162673000  General examination of patient (procedure)         NaN   
2  162673000  General examination of patient (procedure)         NaN   
3  162673000  General examination of patient (procedure)         NaN   
4  162673000  General examination of patient (procedure)         NaN   

  REASONDESCRIPTION ENCOUNTERCLASS                                  Id_y  \
0   

In [5]:
# Create a subset of cases with non-empty "REASONDESCRIPTION"
subset_with_reason = df[df['REASONDESCRIPTION'].notna()]

# Get all cases with "REASONDESCRIPTION" data
cases_with_reason = subset_with_reason.sample(n=372, random_state=42)

# Calculate the number of additional cases needed
additional_cases_needed = 10000 - len(cases_with_reason)

# Create a random sample of additional cases from the remaining data
additional_cases = df.drop(cases_with_reason.index).sample(n=additional_cases_needed, random_state=42)

# Concatenate the two subsets to get the final 10,000 cases
final_df = pd.concat([cases_with_reason, additional_cases])

print("Final Dataframe Shape:", final_df.shape)

Final Dataframe Shape: (10000, 26)


### Identify the outcome of interest (in this case, suicide-related events)

In [6]:
df = final_df
#Examine the outcome of interest
print(df['REASONDESCRIPTION'].value_counts())

# Define keywords related to suicide
suicide_keywords = ['suicide', 'self-harm', 'suicidal']

# Create a binary variable 'SuicideRelatedEvent'
df['SuicideRelatedEvent'] = df['REASONDESCRIPTION'].str.lower().str.contains('|'.join(suicide_keywords))

# Fill NaN values with 0 (assuming 0 indicates events that are not related to suicide)
df['SuicideRelatedEvent'] = df['SuicideRelatedEvent'].fillna(0).astype(int)

# Print the 'SuicideRelatedEvent' column
print(df['SuicideRelatedEvent'])

REASONDESCRIPTION
Suicidal deliberate poisoning      227
Attempted suicide - cut/stab        75
Attempted suicide - suffocation     66
Suicide - suffocation                3
Suicide - firearms                   1
Name: count, dtype: int64
327     1
33      1
15      1
314     1
57      1
       ..
6106    0
5563    0
5762    0
1232    0
7642    0
Name: SuicideRelatedEvent, Length: 10000, dtype: int64


### Examine missing data within the database

In [7]:
import numpy as np

# Check missing values in all features
missing_values = df.isnull().sum()

# Filter features with missing values
features_with_missing = missing_values[missing_values > 0]

# Print missing values per feature
print("Missing values per feature:")
print(features_with_missing)

# Calculate the percentage of missing values per feature
percentage_missing = (features_with_missing / len(df)) * 100

# Print the percentage of missing values per feature
print("\nPercentage of missing values per feature:")
print(percentage_missing)

# Check overall percentage of missing values in the entire dataframe
overall_percentage_missing = (df.isnull().sum().sum() / np.product(df.shape)) * 100
print("\nOverall percentage of missing values in the entire dataframe:", overall_percentage_missing)

Missing values per feature:
REASONCODE            9628
REASONDESCRIPTION     9628
DIAGNOSIS2            9971
DIAGNOSIS3           10000
DIAGNOSIS4           10000
DIAGNOSIS5           10000
DIAGNOSIS6           10000
DIAGNOSIS7           10000
DIAGNOSIS8           10000
dtype: int64

Percentage of missing values per feature:
REASONCODE            96.28
REASONDESCRIPTION     96.28
DIAGNOSIS2            99.71
DIAGNOSIS3           100.00
DIAGNOSIS4           100.00
DIAGNOSIS5           100.00
DIAGNOSIS6           100.00
DIAGNOSIS7           100.00
DIAGNOSIS8           100.00
dtype: float64

Overall percentage of missing values in the entire dataframe: 34.31807692307692


### Identify diagnosis codes within the data

In [45]:
diagnosis_columns = ['DIAGNOSIS1', 'DIAGNOSIS2']

for column in diagnosis_columns:
    # Check unique values in the current diagnosis column
    unique_diagnosis_codes = final_df[column].unique()

    # Print the unique diagnosis codes for the current column
    print(f"Unique Diagnosis Codes in {column}:")
    print(unique_diagnosis_codes)

    # Analyze the distribution of diagnosis codes for the current column
    diagnosis_code_distribution = final_df[column].value_counts()
    print(f"\nDiagnosis Code Distribution in {column}:")
    print(diagnosis_code_distribution)
    print("\n" + "="*50 + "\n")

Unique Diagnosis Codes in DIAGNOSIS1:
[410620009 162673000 287185009  33879002 287182007  86849004 287193009]

Diagnosis Code Distribution in DIAGNOSIS1:
DIAGNOSIS1
162673000    4703
410620009    3950
33879002     1280
86849004       43
287185009      14
287182007       9
287193009       1
Name: count, dtype: int64


Unique Diagnosis Codes in DIAGNOSIS2:
[           nan 2.87185009e+08 2.87182007e+08 8.68490040e+07]

Diagnosis Code Distribution in DIAGNOSIS2:
DIAGNOSIS2
86849004.0     19
287185009.0     7
287182007.0     3
Name: count, dtype: int64




### Create income range and age variable from existing data. Save the modified data to a new file (final_data_modified.csv).

In [22]:
import pandas as pd
from datetime import datetime

# Assuming 'INCOME_RESPONSE' is the column related to income
income_column = 'INCOME'

# Define income bins
income_bins = [0, 25000, 50000, 75000, 100000, 150000, float('inf')] 

# Define corresponding labels for the bins
income_labels = ['0-25k', '25k-50k', '50k-75k', '75k-100k', '100k-150k', '150k+']

# Create a new column 'INCOME_RANGE' with the income ranges
df['INCOME_RANGE'] = pd.cut(df[income_column], bins=income_bins, labels=income_labels, include_lowest=True)

# Display the unique income ranges and their distribution
unique_income_ranges = df['INCOME_RANGE'].unique()
income_range_distribution = df['INCOME_RANGE'].value_counts()

print("Unique Income Ranges:")
print(unique_income_ranges)

print("\nIncome Range Distribution:")
print(income_range_distribution)

df['BIRTHDATE'] = pd.to_datetime(df['BIRTHDATE'])

# Calculate age based on the current date
current_date = datetime.now()
df['Age'] = ((current_date - df['BIRTHDATE']) / pd.Timedelta(days=365.25)).astype(int)

# Display the DataFrame with the new 'Age' column
print(df[['BIRTHDATE', 'Age']])

# Now, read the modified DataFrame back from the CSV file
df_modified = pd.read_csv('/Users/SarahA/synthea/final_data_modified.csv')

Unique Income Ranges:
['25k-50k', '150k+', '0-25k', '75k-100k', '100k-150k', '50k-75k']
Categories (6, object): ['0-25k' < '25k-50k' < '50k-75k' < '75k-100k' < '100k-150k' < '150k+']

Income Range Distribution:
INCOME_RANGE
0-25k        1882
25k-50k      1861
100k-150k    1692
150k+        1646
50k-75k      1605
75k-100k     1314
Name: count, dtype: int64
          BIRTHDATE  Age
30981861 2001-09-25   22
58777479 1939-04-03   84
41337713 1967-08-21   56
16070944 1956-02-18   67
72935481 1974-03-23   49
...             ...  ...
22106434 1965-02-17   58
877023   1964-10-03   59
85859185 1967-03-09   56
22733173 1981-10-22   42
66156749 2016-08-22    7

[10000 rows x 2 columns]
